#### Vacation_Search
#### Colin Brineman, M.A.

# Import weather data

In [1]:
# Import dependencies
import pandas as pd
import requests
import gmaps
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

# Load weather data csv file as DataFrame
weather_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")

# Inspect weather data DataFrame
weather_data_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Haines Junction,CA,60.7522,-137.5108,-4.03,93,100,0.51,overcast clouds
1,1,Port Elizabeth,ZA,-33.9180,25.5701,22.28,78,75,7.20,broken clouds
2,2,Tasiilaq,GL,65.6145,-37.6368,-7.91,94,100,5.71,snow
3,3,Bluff,NZ,-46.6000,168.3333,15.70,70,99,3.83,overcast clouds
4,4,Rikitea,PF,-23.1203,-134.9692,25.14,69,35,3.35,scattered clouds
5,5,Bambous Virieux,MU,-20.3428,57.7575,25.68,88,75,0.51,broken clouds
6,6,Kavieng,PG,-2.5744,150.7967,27.63,77,97,1.27,light rain
7,7,Lazaro Cardenas,MX,17.9583,-102.2000,27.96,64,11,4.28,few clouds
8,8,Samarai,PG,-10.6167,150.6667,27.72,76,70,3.88,broken clouds
9,9,Havre-Saint-Pierre,CA,50.2334,-63.5986,-8.67,72,100,6.17,overcast clouds


# Filter weather data

In [2]:
# Prompt user to input minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 20
What is the maximum temperature you would like for your trip? 30


In [3]:
# Filter weather data based on user input
user_df = weather_data_df.loc[(weather_data_df["Max Temp"] >= min_temp) &
                              (weather_data_df["Max Temp"] <= max_temp)]

# Inspect user DataFrame for missing values
user_df.count()

Unnamed: 0             258
City                   258
Country                254
Lat                    258
Lng                    258
Max Temp               258
Humidity               258
Cloudiness             258
Wind Speed             258
Current Description    258
dtype: int64

In [4]:
# Drop rows with empty values
clean_df = user_df.dropna()

# Inspect clean DataFrame
clean_df.count()

Unnamed: 0             254
City                   254
Country                254
Lat                    254
Lng                    254
Max Temp               254
Humidity               254
Cloudiness             254
Wind Speed             254
Current Description    254
dtype: int64

# Find hotels

In [5]:
# Create hotel DataFrame
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Create new column to store hotel names
hotel_df["Hotel Name"] = ""

# Inspect hotel DataFrame
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Port Elizabeth,ZA,22.28,broken clouds,-33.9180,25.5701,
4,Rikitea,PF,25.14,scattered clouds,-23.1203,-134.9692,
5,Bambous Virieux,MU,25.68,broken clouds,-20.3428,57.7575,
6,Kavieng,PG,27.63,light rain,-2.5744,150.7967,
7,Lazaro Cardenas,MX,27.96,few clouds,17.9583,-102.2000,
8,Samarai,PG,27.72,broken clouds,-10.6167,150.6667,
14,Samarinda,ID,21.78,overcast clouds,-0.5000,117.1500,
18,Souillac,MU,25.76,broken clouds,-20.5167,57.5167,
22,Butaritari,KI,27.99,scattered clouds,3.0707,172.7902,
28,Atuona,PF,25.93,light rain,-9.8000,-139.0333,


In [6]:
# Set parameters to search for hotels within 5km
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through hotel DataFrame
for index, row in hotel_df.iterrows():
    
    # Get latitude and longitude from hotel DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # Set up base URL for Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # Make request and retrieve JSON data from search
    hotels = requests.get(base_url, params=params).json()   
    
    # Get first hotel from results and store name
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    # If no hotel found, skip city
    except(IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [7]:
# Inspect hotel DataFrame for missing values
hotel_df.count()

City                   254
Country                254
Max Temp               254
Current Description    254
Lat                    254
Lng                    254
Hotel Name             254
dtype: int64

In [8]:
# Inspect hotel DataFrame for missing values
hotel_df_blank = hotel_df.loc[(hotel_df["Hotel Name"] == "")]
hotel_df_blank.count()

City                   13
Country                13
Max Temp               13
Current Description    13
Lat                    13
Lng                    13
Hotel Name             13
dtype: int64

In [9]:
# Drop blank hotel rows
hotel_df_clean = hotel_df.loc[(hotel_df["Hotel Name"] != "")]
hotel_df_clean.count()

City                   241
Country                241
Max Temp               241
Current Description    241
Lat                    241
Lng                    241
Hotel Name             241
dtype: int64

In [10]:
# Export hotel data as csv file
hotel_df_clean.to_csv("WeatherPy_vacation.csv", index_label="City_ID")

# Create hotel map

In [11]:
# Create info box template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °C</dd>
</dl>
"""

# Iterate over hotel data to create list of hotel info to pass to info box content
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_clean.iterrows()]

# Get latitude and longitude from each row of hotel data and store in a new DataFrame
locations = hotel_df_clean[["Lat", "Lng"]]

# Generate map with hotel markers
locations = hotel_df_clean[["Lat", "Lng"]]
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call map figure
fig

Figure(layout=FigureLayout(height='420px'))